In [1]:
import pandas as pd
import numpy as np
import os,glob
import cv2
import scipy.io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import heapq

In [2]:
from keras.applications.vgg16 import VGG16,preprocess_input
from keras.layers import Flatten, Dense, Input,concatenate
from keras.layers import Activation, Dropout, GlobalAveragePooling2D,GlobalMaxPooling2D
from keras.models import Model
from keras.models import Sequential
import scipy.spatial.distance as dist
from keras.optimizers import Adam,SGD
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from keras.layers import MaxPooling2D,Conv2D
from keras.utils.np_utils import to_categorical

C:\Users\admin\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [16]:
def lencode(dataset):
    data1 = le.fit_transform(dataset)
    data = to_categorical(data1,len(dataset))# len(x_train))
    return data

def get_feature_vector(img):
    img1 = cv2.resize(img, (imgsize,imgsize))
    feature_vector = output_layer.predict(img1.reshape(1,imgsize, imgsize, 3))
    return feature_vector

def retriev_top5(pred_y):
    top2,top3,top4,top5 =[],[],[],[]
    yhat = pred_y
    for i in range(len(yhat)):
        pred = np.argmax(yhat[i], out=None)   # The class1 belong to 
        for n in range(2,6):
            x = heapq.nlargest(n, yhat[i], key=None)
            for j in range(0,len(x)):
                arr = np.where(yhat==x[j])
            if n==2:
                if len(arr[1])>1:
                    top2.append(int(arr[1][0]))
                else:
                    top2.append(int(arr[1]))
            elif n==3:
                if len(arr[1])>1:
                    top3.append(int(arr[1][0]))
                else:
                    top3.append(int(arr[1]))
            
            elif n==4:
                if len(arr[1])>1:
                    top4.append(int(arr[1][0]))
                else:
                    top4.append(int(arr[1]))
            
            elif n==5:
                if len(arr[1])>1:
                    top5.append(int(arr[1][0]))
                else:
                    top5.append(int(arr[1]))
    top1 = y_pred
    top2 = le.inverse_transform(top2)
    top3 = le.inverse_transform(top3)
    top4 = le.inverse_transform(top4)
    top5 = le.inverse_transform(top5)
    return top1,top2,top3,top4,top5

def calculate_similarity(vector1, vector2):
    return 1-dist.cosine(vector1, vector2)

def get_class(df2,y_iname):
    classes=[]
    for i in range(len(y_true)):
        for j in range(len(df2)):
            if y_iname[i]==df2['imagelabel'][j]:
                cls = df2['class'][j]
                classes.append(cls)
    return classes

def find_similarity(origtest, testtop):
    values=[]
    for i in range(len(origtest)):
        imagepath1 = os.path.join(path,origtest[i])
        imagepath2 = os.path.join(path,testtop[i])
        im1 = cv2.imread(imagepath1)
        im2 =cv2.imread(imagepath2)
        f1 = get_feature_vector(im1)
        f2 =get_feature_vector(im2)
        cs = calculate_similarity(f1,f2)
        values.append(round(cs,2))
    return values

In [3]:
imgsize=100
loadweight = r'F:\samsung_ip_dataset\vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [4]:
basemodel = VGG16(weights=loadweight, include_top=False, input_shape = (imgsize, imgsize, 3))
#basemodel.summary()
for layer in basemodel.layers:
    layer.trainable = False
    
for layer in basemodel.layers:
    print(layer,layer.trainable)

# # build top model
x = Flatten(name='flatten')(basemodel.output)
x = Dropout(0.8)(x)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dropout(0.6)(x)
x = Dense(2048, activation='relu', name='fc2')(x)

# stitch together
vgg16 = Model(inputs= basemodel.input, outputs=x)

# # inspect
vgg16.summary()

<keras.engine.input_layer.InputLayer object at 0x000001B126906A90> False
<keras.layers.convolutional.Conv2D object at 0x000001B126906F98> False
<keras.layers.convolutional.Conv2D object at 0x000001B126906E10> False
<keras.layers.pooling.MaxPooling2D object at 0x000001B126917898> False
<keras.layers.convolutional.Conv2D object at 0x000001B126917F60> False
<keras.layers.convolutional.Conv2D object at 0x000001B1269583C8> False
<keras.layers.pooling.MaxPooling2D object at 0x000001B126971160> False
<keras.layers.convolutional.Conv2D object at 0x000001B126971E80> False
<keras.layers.convolutional.Conv2D object at 0x000001B1269A3208> False
<keras.layers.convolutional.Conv2D object at 0x000001B1269BD2E8> False
<keras.layers.pooling.MaxPooling2D object at 0x000001B1269D0240> False
<keras.layers.convolutional.Conv2D object at 0x000001B1269D0E80> False
<keras.layers.convolutional.Conv2D object at 0x000001B126A0A208> False
<keras.layers.convolutional.Conv2D object at 0x000001B126A232E8> False
<ker

In [5]:
output_layer = Model(inputs=vgg16.input, outputs=vgg16.get_layer('fc2').output)

In [6]:
#########n Get image data

In [6]:
path= r'F:\samsung_ip_dataset\Batch1'
X=[]
imgsize = 100
for image_name in glob.glob(os.path.join(path, '*.jpg')):
    img = cv2.imread(image_name)
    img = cv2.resize(img,(imgsize,imgsize))
    X.append(np.array(img))

In [7]:
X = np.array(X)
X=X/255
print(len(X))
type(X)

2068


numpy.ndarray

In [8]:
####    Extract Image feeatures

In [9]:
feat = basemodel.predict(X, batch_size=32)

In [10]:
x_features = feat
feat.shape

(2068, 3, 3, 512)

Loading Class Label data

In [11]:
mat = scipy.io.loadmat('imagelabels.mat')
labels = mat['labels']
#print(labels)
pds = pd.Series(data=labels[0])
df = pd.DataFrame(pds,index = list(range(0,len(pds))))
df.head()

,0
0,77
1,77
2,77
3,77
4,77


In [12]:
df2 = pd.DataFrame(index = list(range(len(df[:len(X)]))))
df2['class'] = df[:len(X)].reset_index(drop=True)
n = len(df2['class'].unique())
print('num of classes for batch2: ', len(df2['class'].unique()))
print(len(df2))
df2.tail()

num of classes for batch2:  13
2068


,class
2063,80
2064,80
2065,80
2066,80
2067,80


Loading Image Label data

In [17]:
le = LabelEncoder()

In [18]:
names = os.listdir(path)

In [19]:
ynames = lencode(names)
ynames

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [20]:
df2['imagelabel'] = names

In [21]:
df2.tail()

,class,imagelabel
2063,80,image_02064.jpg
2064,80,image_02065.jpg
2065,80,image_02066.jpg
2066,80,image_02067.jpg
2067,80,image_02068.jpg


In [22]:
# Train and test set

In [23]:
x_train,x_rest,y_train,y_rest = train_test_split(x_features,ynames,test_size=0.40)

print('number of train images:', len(x_train))#,len(x_test))
#print(len(y_train), len(y_test))

x_val,x_test,y_val, y_test = train_test_split(x_rest,y_rest,test_size = 0.50)

print('number of val and test images:',len(x_val),len(x_test))
#print(len(y_val), len(y_test))


number of train images: 1240
number of val and test images: 414 414


In [24]:
x_train.shape[1:]

(3, 3, 512)

Build a sequential model to match features


In [25]:
len(x_features)

2068

In [29]:
model_transfer = Sequential()
model_transfer.add(GlobalAveragePooling2D(input_shape=x_train.shape[1:]))
model_transfer.add(Dropout(0.2))
model_transfer.add(Dense(1024, activation='relu'))
model_transfer.add(Dense(len(ynames), activation='softmax'))
model_transfer.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_2 ( (None, 512)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_4 (Dense)              (None, 2068)              2119700   
Total params: 2,645,012
Trainable params: 2,645,012
Non-trainable params: 0
_________________________________________________________________


In [30]:
model_transfer.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [31]:
history = model_transfer.fit(x_train, y_train, batch_size=100, epochs=100, validation_data=(x_val, y_val))

Train on 1240 samples, validate on 414 samples
Epoch 1/100
1240/1240 [==============================] - 2s 2ms/step - loss: 7.9227 - acc: 0.0000e+00 - val_loss: 7.9063 - val_acc: 0.0000e+00
Epoch 2/100
1240/1240 [==============================] - 2s 1ms/step - loss: 7.4335 - acc: 0.0048 - val_loss: 8.1572 - val_acc: 0.0000e+00
Epoch 3/100
1240/1240 [==============================] - 2s 1ms/step - loss: 7.1464 - acc: 0.0121 - val_loss: 10.1130 - val_acc: 0.0000e+00
Epoch 4/100
1240/1240 [==============================] - 2s 1ms/step - loss: 6.8498 - acc: 0.0274 - val_loss: 10.6833 - val_acc: 0.0000e+00
Epoch 5/100
1240/1240 [==============================] - 2s 1ms/step - loss: 6.4031 - acc: 0.0694 - val_loss: 11.3810 - val_acc: 0.0000e+00
Epoch 6/100
1240/1240 [==============================] - 2s 1ms/step - loss: 5.9057 - acc: 0.1282 - val_loss: 12.6642 - val_acc: 0.0000e+00
Epoch 7/100
1240/1240 [==============================] - 2s 1ms/step - loss: 5.3436 - acc: 0.1855 - val_loss: 1

Epoch 59/100
1240/1240 [==============================] - 2s 1ms/step - loss: 0.0435 - acc: 0.9944 - val_loss: 16.0418 - val_acc: 0.0000e+00
Epoch 60/100
1240/1240 [==============================] - 1s 1ms/step - loss: 0.0504 - acc: 0.9960 - val_loss: 16.0409 - val_acc: 0.0000e+00
Epoch 61/100
1240/1240 [==============================] - 2s 1ms/step - loss: 0.0525 - acc: 0.9927 - val_loss: 16.0524 - val_acc: 0.0000e+00
Epoch 62/100
1240/1240 [==============================] - 1s 1ms/step - loss: 0.0487 - acc: 0.9935 - val_loss: 16.0525 - val_acc: 0.0000e+00
Epoch 63/100
1240/1240 [==============================] - 2s 1ms/step - loss: 0.0657 - acc: 0.9903 - val_loss: 16.0430 - val_acc: 0.0000e+00
Epoch 64/100
1240/1240 [==============================] - 2s 1ms/step - loss: 0.0513 - acc: 0.9919 - val_loss: 16.0385 - val_acc: 0.0000e+00
Epoch 65/100
1240/1240 [==============================] - 2s 2ms/step - loss: 0.0405 - acc: 0.9960 - val_loss: 16.0385 - val_acc: 0.0000e+00
Epoch 66/100


In [32]:
yhat = model_transfer.predict(x_test)
print(len(yhat[12]))
heapq.nlargest(3,yhat[12])

2068


[0.32546306, 0.3136689, 0.16879745]

In [33]:
ytmax=np.argmax(y_test, axis=-1, out=None)
y_true = le.inverse_transform(ytmax)
y_true[0]

'image_00237.jpg'

In [34]:
yhatmax=np.argmax(yhat,axis=1, out=None)
y_pred = le.inverse_transform(yhatmax)
y_pred[0]

'image_01033.jpg'

In [35]:
df2.tail()

,class,imagelabel
2063,80,image_02064.jpg
2064,80,image_02065.jpg
2065,80,image_02066.jpg
2066,80,image_02067.jpg
2067,80,image_02068.jpg


In [36]:
top1,top2,top3,top4,top5 = retriev_top5(yhat)

Class Labeling

In [37]:
classt = get_class(df2, y_true)
#print(classt)
class1 = get_class(df2, top1)
#print(class1)
class2 = get_class(df2, top2)
class3 = get_class(df2, top3)
class4 = get_class(df2, top4)
class5 = get_class(df2, top5)
#print(class2)

Similarity Calculation

In [38]:
cs1 = find_similarity(y_true,top1)
cs2 = find_similarity(y_true,top2)
cs3 = find_similarity(y_true,top3)
cs4 = find_similarity(y_true,top4)
cs5 = find_similarity(y_true,top5)

In [39]:
print("\nAccuracy on first retrieved set: ", round(accuracy_score(classt,class1)*100,2),'%')   # Classes
print("\nAccuracy on second retrieved set: ", round(accuracy_score(classt,class2)*100,2),'%')
print("\nAccuracy on third retrieved set: ", round(accuracy_score(classt,class3)*100,2),'%')
print("\nAccuracy on fourth retrieved set: ", round(accuracy_score(classt,class4)*100,2),'%')
print("\nAccuracy on fivth retrieved set: ", round(accuracy_score(classt,class5)*100,2),'%')


Accuracy on first retrieved set:  67.63 %

Accuracy on second retrieved set:  60.63 %

Accuracy on third retrieved set:  57.25 %

Accuracy on fourth retrieved set:  51.21 %

Accuracy on fivth retrieved set:  51.93 %


In [40]:
f_df = pd.DataFrame(index = list(range(len(y_true))))
f_df['QueryImageName'] = y_true
f_df['QueryImageClass'] = classt
f_df['Top1 ImgCls'] = class1
f_df['Top2 ImgCls'] = class2
f_df['Top3 ImgCls'] = class3
f_df['Top4 ImgCls'] = class4
f_df['Top5 ImgCls'] = class5

In [43]:
f_df['Top1 similaritys'] = cs1
f_df['Top2 similaritys'] = cs2
f_df['Top3 similaritys'] = cs3
f_df['Top4 similaritys'] = cs4
f_df['Top5 similaritys'] = cs5

In [44]:
f_df.tail()

,QueryImageName,QueryImageClass,Top1 ImgCls,Top2 ImgCls,Top3 ImgCls,Top4 ImgCls,Top5 ImgCls,Top1 similaritys,Top2 similaritys,Top3 similaritys,Top4 similaritys,Top5 similaritys
409,image_01054.jpg,46,46,46,46,46,82,0.62,0.63,0.65,0.64,0.59
410,image_01940.jpg,78,78,78,78,78,78,0.80,0.78,0.76,0.82,0.80
411,image_01474.jpg,51,51,51,51,51,51,0.63,0.68,0.75,0.69,0.79
412,image_00461.jpg,88,78,78,78,82,78,0.81,0.78,0.79,0.71,0.82
413,image_01564.jpg,44,44,51,51,44,44,0.67,0.72,0.73,0.72,0.69


In [978]:
f_df.to_csv('Result.csv')